Code taken from: https://gist.github.com/ErikBjare/11092794

Need to start up MongoDB in terminal (after installing) with "mongod" 

In [1]:
from datetime import datetime, timedelta

import pymongo
from pymongo import MongoClient

In [2]:
client = MongoClient("localhost", 27017)
db = client.test_database

In [4]:
db.create_collection("test")

Collection(Database(MongoClient('localhost', 27017), u'test_database'), u'test')

In [5]:
db.collection_names()

[u'system.indexes', u'test']

In [6]:
db.test.insert({"users": [{"name": "Erik Bjäreholt", "nick": "erb"}]})
db.test.find_one({"users": [{"name": "Erik Bjäreholt", "nick": "erb"}]})

{u'_id': ObjectId('562e44a14ea62f2eb01f8e64'),
 u'users': [{u'name': u'Erik Bj\xe4reholt', u'nick': u'erb'}]}

In [7]:
db.drop_collection("test")
db.collection_names()

[u'system.indexes']

In [8]:
db.drop_collection("lifelogger-users")
db.drop_collection("lifelogger-sheets")

In [9]:
users = db.create_collection("lifelogger-users")
sheets = db.create_collection("lifelogger-sheets")

In [10]:
user_erb = users.insert({"username": "erb", "name": "Erik Bjäreholt", "email": "erik.bjareholt@gmail.com"})
user_tester = users.insert({"username": "tester", "name": "Test McTest", "email": "tester@example.com"})

In [11]:
user_erb = users.find_one({"username": "erb"})

In [15]:
user_erb

{u'_id': ObjectId('562e45024ea62f2eb01f8e65'),
 u'email': u'erik.bjareholt@gmail.com',
 u'name': u'Erik Bj\xe4reholt',
 u'username': u'erb'}

In [12]:
class Sheet():
    """Baseclass for all logging-sheets"""
    
    def __init__(self, user, year=None, month=None):
        documents = sheets.find({"username": user["username"]})
        if year and month:
            d = datetime(year, month, 1)
        elif year or month:
            raise ValueError("You must specify both year and month, or neither.")
        else:
            d = datetime.now()
        found = False
        for document in documents:
            if document and document["year"] == d.year and document["month"] == d.month:
                found = True
                break
        if not found:
            document = {"username": user["username"],
                        "columns": {"mood": {"08:00": [], "12:00": [], "20:00": []},
                                    "productivity": {"08:00": [], "12:00": [], "20:00": []},
                                    "exercise": {"running": [], "gym": [], "home": []}},
                        "year": d.year,
                        "month": d.month}
            sheets.insert(document)
        self.data = document
            
    def __repr__(self):
        return str(self.data)

In [13]:
sheet_erb = Sheet(user_erb)
sheet_erb_2013_1 = Sheet(user_erb, year=2013, month=1)

In [14]:
for sheet in sheets.find():
    print(sheet)

{u'username': u'erb', u'_id': ObjectId('562e45324ea62f2eb01f8e67'), u'year': 2015, u'columns': {u'productivity': {u'12:00': [], u'20:00': [], u'08:00': []}, u'mood': {u'12:00': [], u'20:00': [], u'08:00': []}, u'exercise': {u'home': [], u'running': [], u'gym': []}}, u'month': 10}
{u'username': u'erb', u'_id': ObjectId('562e45324ea62f2eb01f8e68'), u'year': 2013, u'columns': {u'productivity': {u'12:00': [], u'20:00': [], u'08:00': []}, u'mood': {u'12:00': [], u'20:00': [], u'08:00': []}, u'exercise': {u'home': [], u'running': [], u'gym': []}}, u'month': 1}


Check out the tutorial here: http://api.mongodb.org/python/current/tutorial.html